# Natural Language Processing

In [1]:
# Importing TF and checking the version
import tensorflow as tf

print(tf.__version__)

2.10.1


In [2]:
# Importing helper functions
from DanielBourke_HelperFunctions import create_tensorboard_callback, plot_loss_curves, compare_historys

### Analysing text dataset

In [3]:
# Loading data
import pandas as pd

train_df = pd.read_csv("NLP_text/train.csv")
test_df = pd.read_csv("NLP_text/test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
# Shuffling training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [5]:
# Checking test dataframe
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
# Checking number of training records
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [7]:
# Checking total number of samples
len(train_df), len(test_df)

(7613, 3263)

In [8]:
# Visualising random samples
import random

random_index = random.randint(0, len(train_df) - 5)

for row in train_df_shuffled[["text", "target"]][random_index : random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text:\n{text}")
    print("---")

Target: 1 (real disaster)
Text:
#Metepec #Mexico - ?NIGHT DISASTER?...E(Oficial) @ #NitClub #mÌ¼sica #mÌ¼sica http://t.co/WTfJF9jjzs
---
Target: 1 (real disaster)
Text:
@alextucker VOLCANO BOWL DRINK
---
Target: 0 (not real disaster)
Text:
The 8-Minute Fat-Burning Routine ThatÛªs Also Really Fun http://t.co/g2h7xNecD8 #fat weightless # fatburning #burnfat #skinny #workout
---
Target: 0 (not real disaster)
Text:
Factors to Consider When Hiring a Demolition Company nOxDV
---
Target: 0 (not real disaster)
Text:
#TBT Remember that time Patrick Kane attacked a cab driver over .20
---


### Creating validation data

In [9]:
# Splitting dataset
from sklearn.model_selection import train_test_split

train_sentences, val_sentences, train_labels, val_labels = train_test_split(
    train_df_shuffled["text"].to_numpy(),
    train_df_shuffled["target"].to_numpy(),
    test_size=0.1, # Allocating 10% to validation data
    random_state=42
)

In [10]:
# Checking dataset length
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [11]:
# Checking the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

### Converting text to numbers

In [12]:
# Using text vectorisation
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens=None,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    ngrams=None,
    output_mode="int",
    output_sequence_length=None,
    pad_to_max_tokens=False
)

In [13]:
# Finding the average number of tokens
round(sum([len(i.split()) for i in train_sentences]) / len(train_sentences))

15

In [14]:
# Setting up text vectorisation variables
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    output_mode="int",
    output_sequence_length=max_length
)

In [15]:
# Fitting text vectorisation to the training dataset
text_vectorizer.adapt(train_sentences)

In [16]:
# Create a sample sentnence and tokenise it
sample_sentence = "There is a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int64)>

In [17]:
# Tokenising random sentence from the training set
random_sentence = random.choice(train_sentences)
print(
    f"Original text:\n{random_sentence},\ntokenised version:\n{text_vectorizer([random_sentence])}"
)

Original text:
@RosemaryTravale Do we all use the same weapon? 'cause we might be screwed XD,
tokenised version:
[[8978   68   46   44  447    2  726  263  257   46  843   21 8730 1916
     0]]


In [18]:
# Getting unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]

len(words_in_vocab), top_5_words, bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

### Creating Embedding layer

In [19]:
# Defining the layer
from tensorflow.keras import layers

embedding = layers.Embedding(
    input_dim=max_vocab_length,
    output_dim=128,
    input_length=max_length
)
embedding

In [20]:
# Get a random sentence
random_sentence = random.choice(train_sentences)
print(
    f"Original text:\n{random_sentence},\nembedded version:"
)

# Embed random sentence
embed_sentence = embedding(text_vectorizer([random_sentence]))
embed_sentence

Original text:
@CouncilSCC it does say hailstorm,
embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.01347689,  0.01081227, -0.01314595, ...,  0.01134624,
         -0.03508957,  0.01095806],
        [-0.04150762, -0.02875059, -0.02448224, ..., -0.02722785,
          0.01106387,  0.03568279],
        [-0.00855424, -0.0311479 , -0.00529737, ...,  0.01986209,
         -0.04528741,  0.02708408],
        ...,
        [ 0.03131826, -0.01514022,  0.02296391, ...,  0.0005614 ,
          0.03330268, -0.02141659],
        [ 0.03131826, -0.01514022,  0.02296391, ...,  0.0005614 ,
          0.03330268, -0.02141659],
        [ 0.03131826, -0.01514022,  0.02296391, ...,  0.0005614 ,
          0.03330268, -0.02141659]]], dtype=float32)>

In [21]:
# Checking single token's embedding
embed_sentence[0][0], embed_sentence[0][0].shape, random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.01347689,  0.01081227, -0.01314595,  0.02223697, -0.02041599,
        -0.00706041,  0.0452088 , -0.03344657, -0.04105613,  0.00650709,
         0.0163715 , -0.02625236, -0.0369825 ,  0.00570083, -0.00296791,
        -0.0094461 ,  0.00617895, -0.01487704, -0.03923587, -0.01148658,
        -0.01263583, -0.01754203, -0.01623712,  0.00919318, -0.02260126,
        -0.03906111, -0.02007184, -0.03806884, -0.02933215,  0.00250695,
         0.00178555,  0.03941614, -0.01576471, -0.02000595, -0.00528374,
        -0.02230659, -0.00863902,  0.00590007, -0.04030498, -0.01158222,
        -0.0450322 , -0.0207773 , -0.01431553,  0.03859941, -0.03628879,
         0.03357115,  0.04113933,  0.04809996,  0.03796258, -0.0056957 ,
         0.02898171,  0.00693237, -0.02856143,  0.04935229, -0.02329591,
         0.04564562, -0.0317139 , -0.04288054,  0.03877959,  0.03154181,
         0.02652296,  0.00544713,  0.03085067, -0.03547785,  0.01919732,
  

### Building base model

In [22]:
# Using SKLearn to build base model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenisation and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidfVectorizer()), # Convert words to numbers
    ("clf", MultinomialNB()) # Model the text
])

# Fit the pipeline to training data
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [23]:
# Evaluating baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Achieved accuracy: {baseline_score * 100:.2f}%")

Achieved accuracy: 79.27%


In [24]:
# Making predictions
baseline_predictions = model_0.predict(val_sentences)
baseline_predictions[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

### Function to evaluate model performance

In [25]:
# Importing SKLearn functions
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Developing function to evaluate accuracy, precision, recall and F1 scor
def calculate_results(y_true, y_pred):
    """
    Evaluate binary classification model
    """
    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calculate model precision, recall and F1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
    model_results = {
        "accuracy": model_accuracy,
        "precision": model_precision,
        "recall": model_recall,
        "f1": model_f1
    }
    return model_results

In [26]:
# Getting baseline results
baseline_results = calculate_results(
    y_true=val_labels,
    y_pred=baseline_predictions
)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: a simple dense model

In [27]:
# Creating tensorboard callback
from DanielBourke_HelperFunctions import create_tensorboard_callback

# Creating log directory
SAVE_DIR = "model_logs"

In [28]:
# Building model with Functional API
from tensorflow.keras import layers

inputs = layers.Input(shape=(1, ), dtype=tf.string)
x = text_vectorizer(inputs) # Turn the input text into numbers
x = embedding(x) # Create embedding of numberised inputs
x = layers.GlobalAveragePooling1D()(x) # Condence the feature vector
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [29]:
# Getting model summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [30]:
# Compiling model
model_1.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [31]:
# Fitting the model
history_1 = model_1.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_1_dense"
    )]
)

Saving TensorBoard log files to: model_logs/model_1_dense/20230114-161732
Epoch 1/5
215/215 [==============================] - 9s 32ms/step - loss: 0.6130 - accuracy: 0.6895 - val_loss: 0.5352 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.4412 - accuracy: 0.8210 - val_loss: 0.4749 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3476 - accuracy: 0.8605 - val_loss: 0.4555 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.2852 - accuracy: 0.8908 - val_loss: 0.4640 - val_accuracy: 0.7848
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2380 - accuracy: 0.9124 - val_loss: 0.4751 - val_accuracy: 0.7756


In [32]:
# Checking the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4751 - accuracy: 0.7756


[0.47509005665779114, 0.7755905389785767]

In [33]:
# Making predictions
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 4ms/step


(762, 1)

In [34]:
# First 10 predictions
model_1_pred_probs[:10]

array([[0.4217687 ],
       [0.73004496],
       [0.9975582 ],
       [0.16141163],
       [0.12978622],
       [0.94919074],
       [0.9181515 ],
       [0.9933853 ],
       [0.96926194],
       [0.33147788]], dtype=float32)

In [35]:
# Converting model predictions to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [36]:
# Calculating model 1 results
model_1_results = calculate_results(
    y_true=val_labels,
    y_pred=model_1_preds
)
model_1_results

{'accuracy': 77.55905511811024,
 'precision': 0.7772070861555818,
 'recall': 0.7755905511811023,
 'f1': 0.7736182129212565}

In [37]:
# Retrieving baseline results
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [38]:
# Comparing model results
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

### Visualising learned embeddings

In [39]:
# Getting vocabulary from the text vectorisation layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [40]:
# Checking model 1 details
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [41]:
# Getting the weights
embed_weights = model_1.get_layer("embedding").get_weights()
embed_weights, embed_weights[0].shape

([array([[ 0.015783  , -0.00334908,  0.03756675, ...,  0.01565457,
           0.01792214, -0.03679121],
         [-0.02315077,  0.01504393, -0.00547965, ...,  0.01842208,
          -0.04491741,  0.0018163 ],
         [-0.0224451 ,  0.00997871, -0.00996   , ...,  0.00733669,
          -0.04458904, -0.00248135],
         ...,
         [ 0.03404609,  0.0221143 ,  0.02656981, ..., -0.04279038,
          -0.03772571, -0.00165086],
         [-0.02283798,  0.03905335,  0.0754346 , ...,  0.03326756,
          -0.02747221, -0.04922141],
         [-0.05063306,  0.05983922,  0.03132253, ...,  0.03882468,
          -0.08314517, -0.04567678]], dtype=float32)],
 (10000, 128))

In [42]:
# Creating embedding files (sample from TF Word Embedding documentation)
import io

out_v = io.open("vectors.tsv", "w", encoding="utf-8")
out_m = io.open("metadata.tsv", "w", encoding="utf-8")

for index, word in enumerate(words_in_vocab):
    if index == 0:
        continue # Skip 0, as it's a padding
    vec = embed_weights[0][index]
    out_v.write("\t".join([str(x) for x in vec]) + "\n")
    out_m.write(word + "\n")

out_v.close()
out_m.close()

Generated vectors.tsv and metadata.tsv can be uploaded into Tensorflow Embedding Projector tool at https://projector.tensorflow.org/ to visualise weights in 3D space.

### Model 2 - RNN (Recurrent Neural Network) with LSTM (Long Short-Term Memory)

In [43]:
# CReating an RNN model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
# x = layers.LSTM(units=64, return_sequences=True)(x) # Sequences required when you stack LSTM layers
# print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
# x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

(None, 15, 128)
(None, 64)


In [44]:
# Checking model's summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [45]:
# Compiling the model
model_2.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [46]:
# Fittting the model
history_2 = model_2.fit(
    x=train_sentences,
    y=train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_2_LSTM"
    )]
)

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230114-161800
Epoch 1/5
215/215 [==============================] - 11s 29ms/step - loss: 0.2251 - accuracy: 0.9225 - val_loss: 0.6094 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 6s 26ms/step - loss: 0.1574 - accuracy: 0.9402 - val_loss: 0.6687 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.1277 - accuracy: 0.9531 - val_loss: 0.7074 - val_accuracy: 0.7756
Epoch 4/5
215/215 [==============================] - 5s 21ms/step - loss: 0.1060 - accuracy: 0.9607 - val_loss: 0.8200 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0864 - accuracy: 0.9667 - val_loss: 0.8727 - val_accuracy: 0.7808


In [47]:
# Meking predictions
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 8ms/step


array([[0.03102198],
       [0.9560031 ],
       [0.999782  ],
       [0.09847017],
       [0.00103051],
       [0.9991949 ],
       [0.8240903 ],
       [0.999859  ],
       [0.99975246],
       [0.4020007 ]], dtype=float32)

In [48]:
# Converting pred probs to label format
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [49]:
#Calculate model 2 results
model_2_results = calculate_results(
    y_true=val_labels,
    y_pred=model_2_preds
)
model_2_results

{'accuracy': 78.08398950131233,
 'precision': 0.7816731653312017,
 'recall': 0.7808398950131233,
 'f1': 0.7793746882098481}

### Model 3 - RNN with GRU (gated recurrent unit)

In [50]:
# Building the model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.GRU(64, return_sequences=True)(x) # Sequences, if GRU are stuck on each other
x = layers.GRU(64)(x)
# x = layers.Dense(64, activation="relu")(x)
# x = layers.GlobalAveragePooling1D()(x) # If we want to consolidate GRU layer with sequences
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

In [51]:
# Getting model summary
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [52]:
# Compiling the model
model_3.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [53]:
# Fitting the model
history_3 = model_3.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_3_GRU"
    )]
)

Saving TensorBoard log files to: model_logs/model_3_GRU/20230114-161833
Epoch 1/5
215/215 [==============================] - 8s 23ms/step - loss: 0.1577 - accuracy: 0.9390 - val_loss: 0.7244 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0817 - accuracy: 0.9680 - val_loss: 0.7984 - val_accuracy: 0.7730
Epoch 3/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0717 - accuracy: 0.9737 - val_loss: 0.9451 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0576 - accuracy: 0.9758 - val_loss: 1.1389 - val_accuracy: 0.7782
Epoch 5/5
215/215 [==============================] - 4s 20ms/step - loss: 0.0499 - accuracy: 0.9780 - val_loss: 1.4163 - val_accuracy: 0.7651


In [54]:
# Meking predictions
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs

24/24 [==============================] - 1s 6ms/step


array([[3.08835559e-04],
       [7.95614719e-01],
       [9.99934494e-01],
       [1.38800200e-02],
       [8.55540420e-05],
       [9.99913275e-01],
       [9.91664350e-01],
       [9.99975920e-01],
       [9.99946177e-01],
       [9.69961941e-01],
       [1.96265537e-04],
       [9.95267332e-01],
       [1.75115973e-04],
       [2.09964991e-01],
       [9.34759591e-05],
       [5.71150158e-04],
       [2.42611219e-04],
       [2.58447835e-04],
       [2.17885850e-03],
       [9.99916077e-01],
       [9.99982059e-01],
       [4.79268674e-05],
       [9.99823987e-01],
       [5.38544846e-04],
       [9.99949515e-01],
       [9.99961197e-01],
       [4.15697577e-04],
       [6.78526005e-04],
       [1.28086089e-04],
       [5.46225369e-01],
       [9.99759257e-01],
       [1.99859249e-04],
       [2.69128545e-03],
       [6.82729646e-04],
       [9.97484922e-01],
       [1.45861760e-01],
       [9.99923706e-01],
       [6.11586608e-02],
       [7.71175465e-03],
       [9.99959648e-01],


In [55]:
# Converting model 3 pred probs to label format
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [56]:
# Calculate model 3 results
model_3_results = calculate_results(
    y_true=val_labels,
    y_pred=model_3_preds
)
model_3_results

{'accuracy': 76.50918635170603,
 'precision': 0.766006698423534,
 'recall': 0.7650918635170604,
 'f1': 0.7632813381351602}

### Model 4 - Bidirectional RNN

In [57]:
# Create a model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_bidir")

In [58]:
# Checking model details
model_4.summary()

Model: "model_4_bidir"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 128)              98816     
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,378,945
N

In [59]:
# Compiling the model
model_4.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [60]:
# Fitting the model
history_4 = model_4.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_4_bidir"
    )]
)

Saving TensorBoard log files to: model_logs/model_4_bidir/20230114-161903
Epoch 1/5
215/215 [==============================] - 11s 30ms/step - loss: 0.1062 - accuracy: 0.9698 - val_loss: 0.9913 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 5s 24ms/step - loss: 0.0538 - accuracy: 0.9777 - val_loss: 1.1466 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 6s 26ms/step - loss: 0.0484 - accuracy: 0.9774 - val_loss: 1.3015 - val_accuracy: 0.7703
Epoch 4/5
215/215 [==============================] - 6s 28ms/step - loss: 0.0485 - accuracy: 0.9787 - val_loss: 1.1624 - val_accuracy: 0.7664
Epoch 5/5
215/215 [==============================] - 6s 28ms/step - loss: 0.0403 - accuracy: 0.9812 - val_loss: 1.2603 - val_accuracy: 0.7703


In [61]:
# Making predictions
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 1s 6ms/step


array([[1.2353349e-03],
       [7.3685712e-01],
       [9.9996543e-01],
       [2.0454343e-01],
       [5.8164722e-05],
       [9.9957860e-01],
       [7.8354985e-02],
       [9.9998742e-01],
       [9.9996710e-01],
       [9.9798030e-01]], dtype=float32)

In [62]:
# Adjusting model's probs to the label format
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [63]:
# Checking results of predictions
model_4_results = calculate_results(
    y_true=val_labels,
    y_pred=model_4_preds
)
model_4_results

{'accuracy': 77.03412073490814,
 'precision': 0.7713700595744248,
 'recall': 0.7703412073490814,
 'f1': 0.768571140634933}

### Model 5 - Conv1D

In [64]:
# Testing embedding layer
from tensorflow.keras import layers

embedding_test = embedding(text_vectorizer("This is a test sentence"))
# print(embedding_test.shape)
embedding_test = tf.expand_dims(embedding_test, axis=0)
# print(embedding_test.shape)
conv_1d = layers.Conv1D(
    filters=32,
    kernel_size=5,
    activation="relu",
    padding="valid"
)
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 11, 32]), TensorShape([1, 32]))

### Model 5 - 1D Convolutional Network layer

In [70]:
# Creating a model
from tensorflow.keras import layers

inputs = layers.Input(shape=(1,), dtype=tf.string, name="input_layer")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(
    filters=64,
    kernel_size=5,
    strides=1,
    activation="relu",
    padding="valid"
)(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv1d")

In [71]:
# Checking model details
model_5.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_2 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_5 (Dense)             (None, 1)              

In [72]:
# Compiling the model
model_5.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(),
    metrics=["accuracy"]
)

In [74]:
# Fitting the model
history_5 = model_5.fit(
    train_sentences,
    train_labels,
    epochs=5,
    validation_data=(val_sentences, val_labels),
    callbacks=[create_tensorboard_callback(
        dir_name=SAVE_DIR,
        experiment_name="model_5_conv1d"
    )]
)

Saving TensorBoard log files to: model_logs/model_5_conv1d/20230114-235045
Epoch 1/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0439 - accuracy: 0.9807 - val_loss: 1.2497 - val_accuracy: 0.7585
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0415 - accuracy: 0.9816 - val_loss: 1.2930 - val_accuracy: 0.7480
Epoch 3/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0409 - accuracy: 0.9812 - val_loss: 1.3431 - val_accuracy: 0.7441
Epoch 4/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0401 - accuracy: 0.9822 - val_loss: 1.3221 - val_accuracy: 0.7467
Epoch 5/5
215/215 [==============================] - 4s 19ms/step - loss: 0.0414 - accuracy: 0.9810 - val_loss: 1.2909 - val_accuracy: 0.7480
